In [36]:
import pandas as pd
import numpy as np
import random
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import load_model
import keras

In [37]:
def eval_keras(df_name:str, model_name:str, num_to_test:int) -> float:
    # read data
    df = pd.read_csv(df_name)
    X = df.drop('Outcome', axis=1)
    y = df['Outcome']

    # convert data to arrays
    X = np.array(X)
    y = np.array(y)

    # split data into the appropriate groups
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    # scale X data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # load the saved model
    model = load_model(model_name)

    # read the data for the predictions to be compared to
    real_X = pd.read_csv(df_name).drop('Outcome', axis=1)
    real_y = pd.read_csv(df_name)['Outcome']

    # loop through the real data and assess how accurate the models are
    accurate_total = 0
    df_len = len(real_X)
    for i in [random.randint(0, len(real_X)) for _ in range(num_to_test)]:
        try:
            prediction = model.predict(scaler.transform(real_X.iloc[i].values.reshape(1, -1)), verbose=0)
            actual = real_y.iloc[i]
            # confidence = abs(prediction - .5)
            if prediction > .5 and actual == 1:
                accurate_total += 1
            elif prediction < .5 and actual == 0:
                accurate_total += 1
        except:
            pass
    accuracy = accurate_total / num_to_test
    return accuracy

In [38]:
eval_keras("csvs/ml_X.csv", "built_models/keras_ml.h5", 1000)

0.652

In [39]:
eval_keras("csvs/ou_X.csv", "built_models/keras_ou.h5", 1000)

0.512

In [40]:
eval_keras("csvs/pl_X.csv", "built_models/keras_pl.h5", 1000)

0.626

In [41]:
import torch
import torch.nn as nn

def eval_pytorch(df_name: str, model_name: str, num_to_test: int) -> float:
    # Read data
    df = pd.read_csv(df_name)
    X = df.drop('Outcome', axis=1)
    y = df['Outcome']

    # Convert data to arrays
    X = np.array(X)
    y = np.array(y)

    # Split data into the appropriate groups
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    # Scale X data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    class ANN_Model(nn.Module):
        def __init__(self, input_size):
            super(ANN_Model, self).__init__()
            self.fc1 = nn.Linear(input_size, 256)
            self.fc2 = nn.Linear(256, 128)
            self.fc3 = nn.Linear(128, 64)
            self.fc4 = nn.Linear(64, 32)
            self.fc5 = nn.Linear(32, 1)
            self.relu = nn.ReLU()
            self.dropout = nn.Dropout(0.1)
            
        def forward(self, x):
            x = self.relu(self.fc1(x))
            x = self.dropout(x)
            x = self.relu(self.fc2(x))
            x = self.dropout(x)
            x = self.relu(self.fc3(x))
            x = self.dropout(x)
            x = self.relu(self.fc4(x))
            x = self.dropout(x)
            x = self.fc5(x)
            return x
    
    # Load the saved model
    model = ANN_Model(input_size=X_train.shape[1])
    model.load_state_dict(torch.load(model_name))
    device = torch.device("cuda")
    model.to(device)
    model.eval()

    # Loop through the test data and assess accuracy
    accurate_total = 0
    for i in np.random.randint(0, len(X_test), num_to_test):
        input_tensor = torch.tensor(X_test[i], dtype=torch.float32).unsqueeze(0).to(device)
        target_tensor = torch.tensor(y_test[i], dtype=torch.float32).unsqueeze(0).to(device)
        
        # Perform prediction
        output_tensor = model(input_tensor)
        predicted_label = torch.round(torch.sigmoid(output_tensor))
        
        # Compare predicted and actual labels
        if predicted_label == target_tensor:
            accurate_total += 1

    accuracy = accurate_total / num_to_test
    return accuracy

In [42]:
eval_pytorch("csvs/ml_X.csv", "built_models/torch_ml.pth", 1000)

0.93

In [43]:
eval_pytorch("csvs/ou_X.csv", "built_models/torch_ou.pth", 1000)

0.52

In [44]:
eval_pytorch("csvs/pl_X.csv", "built_models/torch_pl.pth", 1000)

0.857

In [46]:
a = eval_keras("csvs/new_ml_X.csv", "built_models/keras_ml.h5", 500)
b = eval_pytorch("csvs/new_ml_X.csv", "built_models/torch_ml.pth", 500)
c = eval_keras("csvs/new_ou_X.csv", "built_models/keras_ou.h5", 500)
d = eval_pytorch("csvs/new_ou_X.csv", "built_models/torch_ou.pth", 500)
e = eval_keras("csvs/new_pl_X.csv", "built_models/keras_pl.h5", 500)
f = eval_pytorch("csvs/new_pl_X.csv", "built_models/torch_pl.pth", 500)

print(a, b, c, d, e, f)

0.512 0.494 0.492 0.462 0.588 0.592
